In [3]:
import gzip, json
import random
import re
import pronouncing
import matplotlib.pyplot as plt
import pandas as pd
from textblob import TextBlob
import nltk

In [4]:
#This just makes a list of all lines of poetry, sans extra whitespace
all_lines =[]
for line in gzip.open("gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

In [5]:
#builds a dictionary that maps words with that rhyming part to the lines of poetry that they're found at the end of.
from collections import defaultdict

by_rhyming_part = defaultdict(lambda: defaultdict(list))

for line in all_lines:
    text = line['s']
    if not(32 < len(text) < 48): # only use lines of uniform lengths
        continue
        # /b indicates word boundaries
        # w+ 'word character' alphanumeric I think
        # W not word character 
        
    match = re.search(r'(\b\w+\b)\W*$', text)
    
    if match:
        last_word = match.group()
        pronunciations = pronouncing.phones_for_word(last_word)
        if len(pronunciations) > 0:
               rhyming_part = pronouncing.rhyming_part(pronunciations[0])
               # group by rhyming phones (for rhymes) and words (to avoid duplicate words)
               by_rhyming_part[rhyming_part][last_word.lower()].append(text)
               
               

In [6]:
lines_frame = pd.DataFrame(by_rhyming_part)

In [7]:
justlines = []
for each in all_lines:
    justlines.append(each['s'])



In [8]:
import string
no_punct = []
for each in justlines:
    punctless = each.strip(string.punctuation)
    no_punct.append(punctless)


### Certain words aren't being caught by my pronunciation checker. Is it 'cause they're old english or something?

In [9]:
#source_word = 'knelling'
#print(source_word.isalpha())
#phones = pronouncing.phones_for_word(source_word)[0] #
#print(phones)
#rhyming_part = pronouncing.rhyming_part(phones)

These words make my prounouncing library dry heave

In [10]:
bad_words =[]
for line in no_punct:
    text = line
    source_word = text.split(' ')[-1]
    try:
        pronouncing.phones_for_word(source_word)[0]
    except IndexError:
        bad_words.append(text.split(' ')[-1])
len(bad_words)    

347149

In [11]:
bad_words = pd.Series(bad_words)
bad_words.sample(5)

116925         Pohyola
301810        senectus
147071           stept
150994          to-day
110663    master-magic
dtype: object

In [14]:
no_punct_no_repeats = set(no_punct) -set(bad_words)

## Analysis of original corpus

In [17]:
corpus = pd.DataFrame(no_punct_no_repeats)
corpus['Line'] = corpus[0]
corpus.drop([0], axis=1, inplace= True)
corpus.head()

,Line
0,And sought her 'custom'd couch. Scarce Peleus ...
1,But never leave their firm intent
2,How gloriously beautiful is earth
3,And by her girdle hung a purse of leather
4,As plain as the land of bread


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# THE MACHINE

In [ ]:
#This whole thing doesn't work super reliably. Some of it has to do with the pronunciation library missing words in my corpus,
#some of it has to do with punctuation. Working on it, but it's not super high priority.
def limirick_machine(tone_param=True):
    #tone_param, sorts sentiment. True = ascending, or sad, false = Descending, or happy
    #Finds you lines that don't need to rhyme
    def initial_line_finder(list_of_existing):
        rand_choice = random.choice(no_punct_no_repeats)
        if rand_choice not in list_of_existing:
            line = rand_choice
            
        while line.split(' ')[-1].isalpha() == False:
            rand_choice = random.choice(no_punct_no_repeats)
            if rand_choice not in list_of_existing:
                line = rand_choice
        
        flag = False
        while flag == False:            
            try:
                
                word_to_rhyme = line.split(' ')[-1]
                phones = pronouncing.phones_for_word(word_to_rhyme)
                phones = phones[0]
                flag= True
                
            except IndexError:  
                
                rand_choice = random.choice(no_punct_no_repeats)
                while line.split(' ')[-1].isalpha() == False:
                    rand_choice = random.choice(no_punct_no_repeats)
                    if rand_choice not in list_of_existing:
                        line = rand_choice
                phones = pronouncing.phones_for_word(word_to_rhyme)
                phones = phones[0]
            
            
            
        
        ###
        
        rhyming_phones = pronouncing.rhyming_part(phones)
        return line, rhyming_phones
    
    # Finds lines that end with similar rhymes.
    def secondary_line_finder(list_of_existing, phones_to_rhyme, cosine_buddy):   
        
        
        rand_choice = ''
        ##############################################
        
        def rhymeme(focus, tone=True):
            def sentimenter(line):
                sentiment= TextBlob(line).sentiment
                subjectivity = sentiment.subjectivity
                return (subjectivity)
                
            line_idx = idx_assign[idx_assign == focus].index[0]
            
            idx_assign['line'] = idx_assign[0]
            idx_assign.drop(inplace=True, axis=1, labels=[0])
            
            sim_array = cos_sim[line_idx]
            
            #makes the cosign dataframe
            
            array_lst = list(enumerate(sim_array))
            sort_lst = sorted(array_lst, reverse=True, key = lambda x: x[1])
            cosign_df = pd.DataFrame(sort_lst)
            #renaming columns
            cosign_df['orig_ind'] = cosign_df[0] 
            cosign_df.drop(inplace=True, axis=1, labels=[0])
            cosign_df['cosign_score'] = cosign_df[1]
            cosign_df.drop(inplace=True, axis=1, labels=[1])
            
            #builds a df with both cosign similarity and sentiment in it
            #merges cosign df and the df with the rhyming lines in it
            co_sentiment_df = idx_assign.merge(cosign_df, left_on='index', right_on='orig_ind')
            co_sentiment_df.drop(inplace=True, axis=1, labels=['index', 'orig_ind'])
            #applies the sentiment analyzer to the givgen ligns
            co_sentiment_df['sentiment'] = co_sentiment_df['line'].apply(sentimenter)
            co_sentiment_df.sort_values(['cosign_score'], ascending=False)
            
            
            #Sorts by cosign
            sorted_cosign = co_sentiment_df.sort_values(['cosign_score'], ascending=False).head(10)
            #Sorts the dataframe by sentiment
            sorted_sentiment = sorted_cosign.sort_values(['sentiment'], ascending=False)
            sorted_sentiment.drop(axis=0, labels=[0], inplace=tone)

            final_phrase = sorted_sentiment['line'].iloc[0]

            return final_phrase
        ##############################################
        
        while rand_choice in list_of_existing or rand_choice =='':
            key = random.choice(list(by_rhyming_part[phones_to_rhyme].keys()))    
            list_of_rhyming_lines = list(by_rhyming_part[phones_to_rhyme][key])

            count_vect = CountVectorizer(ngram_range=(1, 2), stop_words='english')

            to_vectorize = list_of_rhyming_lines
            to_vectorize.append(cosine_buddy)
            
            
            count_matrix = count_vect.fit_transform(to_vectorize)
            cos_sim = cosine_similarity(count_matrix)
            
            rhyming_series = pd.Series(to_vectorize)
            
            

            
            
            
            idx_assign = rhyming_series.reset_index() 
            rand_choice = rhymeme(cosine_buddy, tone_param)
            
            
        return rand_choice
            

    list_of_lines = []
    line1, line1_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line1)
        
    line2 = secondary_line_finder(list_of_lines, line1_phones, list_of_lines[-1])
    list_of_lines.append(line2)
    
    line3, line3_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line3)
    
    line4 = secondary_line_finder(list_of_lines, line3_phones,list_of_lines[-1])
    list_of_lines.append(line4)
    
    line5 = secondary_line_finder(list_of_lines, line1_phones,list_of_lines[-1])
    

    return line1, line2, line3, line4, line5

    
    

In [34]:
line1, line2, line3, line4, line5 = limirick_machine(tone_param=True)
print('  ')
print('  ')
print(line1)
print(line2)
print(line3)
print(line4)
print(line5)

  
  
And winds, austere and pure
"He will not. If one evil we endure
If care were not the waiter
Whilst that vast, breathless amphitheatre
And still, behind me--hieroglyph obscure


In [40]:
# #initial set up.
# temp_limirick_dict= {'line1':[],'line2':[],'line3':[],'line4':[],'line5':[],}
# for i in range(10):
#     line1, line2, line3, line4, line5 = limirick_machine()
#     temp_limirick_dict['line1'].append(line1)
#     temp_limirick_dict['line2'].append(line2)
#     temp_limirick_dict['line3'].append(line3)
#     temp_limirick_dict['line4'].append(line4)
#     temp_limirick_dict['line5'].append(line5)
# basic_limirick_df = pd.DataFrame(temp_limirick_dict)


    
    

In [ ]:
# #increase size
# temp_limirick_dict= {'line1':[],'line2':[],'line3':[],'line4':[],'line5':[],}
# for i in range(10):
#     line1, line2, line3, line4, line5 = limirick_machine()
#     temp_limirick_dict['line1'].append(line1)
#     temp_limirick_dict['line2'].append(line2)
#     temp_limirick_dict['line3'].append(line3)
#     temp_limirick_dict['line4'].append(line4)
#     temp_limirick_dict['line5'].append(line5)
# temp_limirick_df = pd.DataFrame(temp_limirick_dict)
# basic_limirick_df = pd.concat([basic_limirick_df, temp_limirick_df])


In [ ]:
def sentimenter(line):
    sentiment= TextBlob(line).sentiment
    subjectivity = sentiment.subjectivity
    return (subjectivity)
def polarityer(line):
    sentiment= TextBlob(line).sentiment
    polarity = sentiment.polarity
    return (polarity)

# for column in basic_limirick_df:
#     basic_limirick_df[column + '_sentiment'] = basic_limirick_df[column].apply(sentimenter)
#     basic_limirick_df[column + '_polarity'] = basic_limirick_df[column].apply(polarityer)

In [ ]:
#make a wordcloud
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])
textt = " ".join(review for review in basic_limirick_df.line2)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()